#### Classificação para a base de imagens de cachorros e gatos

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image

#### Estrutura da rede neural convolucional

In [2]:
classificador = Sequential()
classificador.add(Conv2D(32, (3,3), input_shape=(64,64, 3), activation='relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size=(2,2)))

classificador.add(Conv2D(32, (3,3), input_shape=(64,64, 3), activation='relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size=(2,2)))

classificador.add(Flatten())

classificador.add(Dense(units=128, activation='relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units=128, activation='relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units=1, activation='softmax'))
classificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#### Gerar imagens com enquadramentos diferentes

In [3]:
gerador_treinamento = ImageDataGenerator(rescale=1./255, rotation_range= 7,horizontal_flip=True, shear_range=0.2,height_shift_range=0.07, zoom_range=0.2)

In [4]:
gerador_teste = ImageDataGenerator(rescale=1./255)

In [5]:
base_treinamento = gerador_treinamento.flow_from_directory('../../data/baseCaesGatos/dataset/training_set', target_size=(64,64), batch_size=32, class_mode='binary')

Found 4000 images belonging to 2 classes.


In [6]:
base_teste = gerador_teste.flow_from_directory('../../data/baseCaesGatos/dataset/test_set', target_size=(64,64), batch_size=32, class_mode='binary')

Found 1000 images belonging to 2 classes.


- rodando poucas epocas e dividindo a base de treinamento entre as epocas porque o custo é muito grande

In [14]:
classificador.fit_generator(base_treinamento, steps_per_epoch=4000/8, epochs=10, validation_data=base_teste, validation_steps=1000/32, use_multiprocessing=True)

Epoch 1/10
500/500 [==============================] - 260s 521ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 9.5308 - val_accuracy: 0.5000
Epoch 2/10
500/500 [==============================] - 732s 1s/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 9.5308 - val_accuracy: 0.5000
Epoch 3/10
500/500 [==============================] - 764s 2s/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
Epoch 4/10
500/500 [==============================] - 261s 522ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 9.5308 - val_accuracy: 0.5000
Epoch 5/10
500/500 [==============================] - 211s 422ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 9.5308 - val_accuracy: 0.5000
Epoch 6/10
500/500 [==============================] - 216s 432ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 13.3431 - val_accuracy: 0.5000
Epoch 7/10
500/500 [==============================] - 282s 563ms/step - loss: 7.6666 - accuracy: 0.5000 - val_loss: 5.7185 - val_

#### Predição de uma imagem de classe da classe

In [15]:
base_treinamento.class_indices

{'cachorro': 0, 'gato': 1}

In [27]:
imagem_teste = image.load_img('../../data/baseCaesGatos/dataset/test_set/gato/cat.3500.jpg', target_size = (64,64))
imagem_teste = image.img_to_array(imagem_teste)

In [28]:
imagem_teste /= 255
imagem_teste = np.expand_dims(imagem_teste, axis = 0)

In [29]:
imagem_teste.shape

(1, 64, 64, 3)

In [30]:
previsao1 = classificador.predict(imagem_teste)

In [35]:
def normalizarPrevisao(previsao):
    resposta = ''
    if previsao < 0.5:
        resposta = 'cachorro'
    else:
        resposta = 'gato'
    return resposta

In [36]:
normalizarPrevisao(previsao1)

'gato'

In [37]:
imagem_teste2 = image.load_img('../../data/baseCaesGatos/dataset/test_set/cachorro/dog.3500.jpg', target_size = (64,64))
imagem_teste2 = image.img_to_array(imagem_teste2) 

In [38]:
imagem_teste2 /= 255
imagem_teste2 = np.expand_dims(imagem_teste2, axis = 0)

In [39]:
previsao2 = classificador.predict(imagem_teste)

In [41]:
normalizarPrevisao(previsao2)

'gato'